In [ ]:
pip install gurobipy

In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('jan.csv')

In [136]:
pd.set_option('display.max_rows', None) 
data=df #[df['DAY_OF_MONTH']==2]

data = data.sort_values('CRS_DEP_TIME')


In [137]:
def convert_hhmm_to_minutes(hhmm):
    hours = hhmm // 100
    minutes = hhmm % 100
    return hours * 60 + minutes
# Convert ScheduledTime column to minutes

data['CRS_ARR_TIME'] = data['CRS_ARR_TIME'].apply(convert_hhmm_to_minutes)
data['CRS_DEP_TIME'] = data['CRS_DEP_TIME'].apply(convert_hhmm_to_minutes)

In [138]:
def convert_minutes_to_hhmm_string(minutes):
    hours = minutes // 60
    remaining_minutes = minutes % 60
    return f"{hours:02}:{remaining_minutes:02}"  # Format as hh:mm


In [139]:
data.columns

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK',
       'OP_UNIQUE_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR',
       'DEST_AIRPORT_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'TAXI_IN',
       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED', 'DIVERTED',
       'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY'],
      dtype='object')

In [169]:
import pandas as pd
from gurobipy import Model, GRB

# Load the dataset
data = pd.read_csv('jan.csv')

# Define sets
arrival_flights = data['OP_CARRIER_FL_NUM'][data['DEST'] == 'ATL'].unique().tolist()
departure_flights = data['OP_CARRIER_FL_NUM'][data['ORIGIN'] == 'ATL'].unique().tolist()

# Create a Gurobi model
model = Model('Flight_Rescheduling')

# Decision Variables
u_plus = model.addVars(arrival_flights, vtype=GRB.INTEGER, name='u_plus')
u_minus = model.addVars(arrival_flights, vtype=GRB.INTEGER, name='u_minus')
v_plus = model.addVars(departure_flights, vtype=GRB.INTEGER, name='v_plus')
v_minus = model.addVars(departure_flights, vtype=GRB.INTEGER, name='v_minus')

T = 288  # Number of time slots
# Binary variables for newly scheduled times
w_arr = model.addVars(arrival_flights, range(T), vtype=GRB.BINARY, name='w_arr')
w_dep = model.addVars(departure_flights, range(T), vtype=GRB.BINARY, name='w_dep')

# Objective Function: Minimize total displacements
model.setObjective(
    sum(u_plus[i] + u_minus[i] for i in arrival_flights) +
    sum(v_plus[j] + v_minus[j] for j in departure_flights)+
    sum((sum(w_arr[i, t] for t in range(T)) * 5) - data[data['OP_CARRIER_FL_NUM'] == i]['CRS_ARR_TIME'].values[0] for i in arrival_flights)+
    sum((sum(w_dep[i, t] for t in range(T)) * 5) - data[data['OP_CARRIER_FL_NUM'] == i]['CRS_DEP_TIME'].values[0] for i in departure_flights),
    GRB.MINIMIZE
)

# Constraints
# Ensure all flights are scheduled (at least one time slot must be used)
for i in arrival_flights:
    model.addConstr(sum(w_arr[i, t] for t in range(T)) >= 1, name=f'initial_arrival_{i}')
    
for j in departure_flights:
    model.addConstr(sum(w_dep[j, t] for t in range(T)) >= 1, name=f'initial_departure_{j}')

#Rescheduled time constraints
for i in arrival_flights:
    model.addConstr(
        sum(w_arr[i, t] for t in range(T))*5  == u_plus[i] - u_minus[i],
        name=f'rescheduled_arrival_{i}'
    )
    
for j in departure_flights:
    model.addConstr(
        sum(w_dep[j, t] for t in range(T))*5  == v_plus[j] - v_minus[j],
        name=f'rescheduled_departure_{j}'
    )
#- data[data['OP_CARRIER_FL_NUM'] == i]['CRS_DEP_TIME'].values[0]
# Capacity constraints
for t in range(T):
    model.addConstr(
        sum(w_arr[i, t] for i in arrival_flights) <= 10,  # Ensure this matches realistic airport operations
        name=f'capacity_arrival_{t}'
    )
    
    model.addConstr(
        sum(w_dep[j, t] for j in departure_flights) <= 15,  # Same as above
        name=f'capacity_departure_{t}'
    )

# Set maximum displacements
model.addConstrs((u_plus[i] <= 60 for i in arrival_flights), name='max_pos_displacement_arrival')
model.addConstrs((u_minus[i] <= 15 for i in arrival_flights), name='max_neg_displacement_arrival')
model.addConstrs((v_plus[j] <= 60 for j in departure_flights), name='max_pos_displacement_departure')
model.addConstrs((v_minus[j] <= 15 for j in departure_flights), name='max_neg_displacement_departure')

# Optimize the model
model.optimize()

# Check the status of the model
if model.status == GRB.OPTIMAL:
    print("Optimal scheduling found:")
    for i in arrival_flights:
        new_schedule = sum(t * w_arr[i, t].X for t in range(T) if w_arr[i, t].X > 0)
        print(f"Flight {i}: New Schedule = {new_schedule}")
    
    for j in departure_flights:
        new_schedule = sum(t * w_dep[j, t].X for t in range(T) if w_dep[j, t].X > 0)
        print(f"Flight {j}: New Schedule = {new_schedule}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14428 rows, 1004270 columns and 3005884 nonzeros
Model fingerprint: 0x022a3941
Variable types: 0 continuous, 1004270 integer (997344 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 12395 rows and 499965 columns (presolve time = 7s) ...
Presolve removed 12395 rows and 499965 columns
Presolve time: 7.44s
Presolved: 2033 rows, 504305 columns, 1006865 nonzeros
Variable types: 0 continuous, 504305 integer (502560 binary)
Found heuristic solution: objective -5067395.000
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barr

In [170]:
# Assuming 'arrival_delay' and 'departure_delay' columns exist in your original dataset
initial_arrival_delay = data[data['DEST'] == 'ATL']['ARR_DELAY'].sum()  # Total initial delay for arrival flights
initial_departure_delay = data[data['ORIGIN'] == 'ATL']['DEP_DELAY'].sum()  # Total initial delay for departure flights

# After optimizing the model, calculate the optimized delays
optimized_arrival_delay = 0
optimized_departure_delay = 0

for i in arrival_flights:
    # Find the first time slot where the flight is scheduled after optimization
    for t in range(T):
        if w_arr[i, t].X > 0:
            optimized_arrival_delay += max(0, 5*t - data[data['OP_CARRIER_FL_NUM'] == i]['CRS_ARR_TIME'].values[0])   # Assuming 5 min intervals

for j in departure_flights:
    # Find the first time slot where the flight is scheduled after optimization
    for t in range(T):
        if w_dep[j, t].X > 0:
            optimized_departure_delay += max(0, 5*t - data[data['OP_CARRIER_FL_NUM'] == j]['CRS_DEP_TIME'].values[0])  # Assuming 5 min intervals

# Calculate total initial and optimized delays
total_initial_delay = initial_arrival_delay + initial_departure_delay
total_optimized_delay = optimized_arrival_delay + optimized_departure_delay

# Calculate reduction in delay
reduction_in_delay = total_initial_delay - total_optimized_delay

# Print results
print(f"Total Initial Delay: {total_initial_delay} minutes")
print(f"Total Optimized Delay: {total_optimized_delay} minutes")
print(f"Reduction in Delay: {reduction_in_delay} minutes")


Total Initial Delay: 553817.0 minutes
Total Optimized Delay: 257199 minutes
Reduction in Delay: 296618.0 minutes


In [171]:
print(f"Percentage reduction in delay:{reduction_in_delay/total_initial_delay*100}%")

Percentage reduction in delay:53.558847055976976%


In [ ]:
# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['Day', 'Flight Number', 'Original Time', 'New Time', 'Type'])

# Get the unique days in the dataset
days_in_month = data['DAY_OF_MONTH'].unique()

# Loop through each day in the month
for day in days_in_month:
    # Filter data for the current day
    day_data = data[data['DAY_OF_MONTH'] == day]
    
    # Loop through each arrival flight for this day
    for i in arrival_flights:
        # Check if the flight exists in the data for the current day
        if i in day_data['OP_CARRIER_FL_NUM'].values:
            # Original arrival time
            original_arrival = day_data[day_data['OP_CARRIER_FL_NUM'] == i]['CRS_ARR_TIME'].values[0]
            # Rescheduled arrival time in minutes
            new_arrival_time = sum(t * 5 for t in range(T) if w_arr[i, t].X > 0)
            
            # Append data to the results DataFrame
            results_df = pd.concat([
                results_df, 
                pd.DataFrame({
                    'Day': [day],
                    'Flight Number': [i],
                    'Original Time': [original_arrival],
                    'New Time': [new_arrival_time],
                    'Type': ['Arrival']
                })
            ], ignore_index=True)
    
    # Loop through each departure flight for this day
    for j in departure_flights:
        # Check if the flight exists in the data for the current day
        if j in day_data['OP_CARRIER_FL_NUM'].values:
            # Original departure time
            original_departure = day_data[day_data['OP_CARRIER_FL_NUM'] == j]['CRS_DEP_TIME'].values[0]
            # Rescheduled departure time in minutes
            new_departure_time = sum(t * 5 for t in range(T) if w_dep[j, t].X > 0)
            
            # Append data to the results DataFrame
            results_df = pd.concat([
                results_df, 
                pd.DataFrame({
                    'Day': [day],
                    'Flight Number': [j],
                    'Original Time': [original_departure],
                    'New Time': [new_departure_time],
                    'Type': ['Departure']
                })
            ], ignore_index=True)

# Display the resulting DataFrame
print(results_df)


In [212]:
results_df.to_csv("rescheduled1.csv")